¿Que hace este script?
* Calcula para cada dispositivo de red: trafico.totales.traficoIN, trafico.totales.traficoOUT', trafico.totales.consumoGB'.
* Calcula para cada site_id: trafico.totales.sitio.traficoIN, trafico.totales.sitio.traficoOUT, trafico.totales.sitio.consumoGB

In [25]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [26]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [27]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### Definiendo indice principal con fecha de hoy

Estos valores se deben ajustar según ambiente. No es automático ya que no hay separación de ambientes

In [28]:
indice = parametros.conteo_centros_index
indice_control = parametros.tableros_mintic_control

### Función para generar JSON compatible con ES

In [29]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede, energía, latitud, longitud, COD_ISO, entre otros).

In [30]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob', 'nombreSede' 
                           , 'energiadesc', 'latitud', 'longitud','COD_ISO','id_Beneficiario']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    print("fecha:",now,"- Error en lectura de datos semilla")
    #exit()
    

Se valida latitud y longitud, se genera campo location y se renombran los campos de semilla

In [31]:
def get_location(x,y='lat'):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None) and (str(x)!=''):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return '4.596389' if y=='lat' else '-74.074639'
    
datos_semilla['latitud'] = datos_semilla['latitud'].apply(lambda x:get_location(x,'lat'))
datos_semilla['longitud'] = datos_semilla['longitud'].apply(lambda x:get_location(x,'lon'))
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['centros.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['centros.location']=datos_semilla['centros.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

datos_semilla = datos_semilla.rename(columns={'nombre_municipio': 'centros.nombreMunicipio'
                                              , 'nombre_departamento' : 'centros.nombreDepartamento'
                                              , 'nombre_centro_pob': 'centros.localidad'
                                              , 'nombreSede' : 'centros.nomCentroDigital'
                                              , 'energiadesc' : 'centros.sistemaEnergia'
                                              , 'COD_ISO' : 'centros.codISO'
                                              , 'id_Beneficiario' : 'centros.idBeneficiario'})
datos_semilla.fillna('', inplace=True)

In [32]:
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["centros.location"]=='')].index)

In [33]:
datos_semilla

,centros.nomCentroDigital,centros.codISO,centros.localidad,site_id,centros.nombreDepartamento,centros.sistemaEnergia,centros.nombreMunicipio,centros.idBeneficiario,centros.location
0,C.E.R LA PAULINA,CO-ANT,LA PAULINA,22154-ZGYO176,ANTIOQUIA,RED INTERCONECTADA,VALDIVIA,22154,"6.7048548,-76.14525469"
1,CENTRO EDUCATIVO RURAL EL CERRO,CO-ANT,EL CERRO,20022-ZGYO177,ANTIOQUIA,RED INTERCONECTADA,FRONTINO,20022,"4.225166667,-73.10716667"
2,C. E. R. MERCEDES YEPES,CO-ANT,VDA. LA MESETA,,ANTIOQUIA,RED INTERCONECTADA,ANORÍ,18787,"6.93848274,-75.151177190"
3,C. E. R. MEDIAS FALDAS,CO-ANT,VDA EL LIMÓN,,ANTIOQUIA,TIENE ENERGÍA,ANORÍ,18802,"7.04797951,-75.19442611"
4,C. E. R. ROSAURA ORTEGA,CO-ANT,BELLAVISTA,,ANTIOQUIA,RED INTERCONECTADA,ANORÍ,18804,"7.06272584,-75.22730972"
...,...,...,...,...,...,...,...,...,...
6942,ESCUELA SANTA ISABEL,CO-VAU,COMUNIDAD SANTA ISABEL,,VAUPÉS,SOLUCIÓN SOLAR,PACOA,67733,"6.39026335,-73.138349330"
6943,ESCUELA RURAL DE SAN LUIS DE PIEDRA ÑI,CO-VAU,COMUNIDAD DE SAN LUIS,,VAUPÉS,SIN INFORMACIÓN,PACOA,70622,"0.08164385,-70.07416384"
6944,ESCUELA RURAL DE SANTA ROSA,CO-VAU,COMUNIDAD DE SANTA ROSA,,VAUPÉS,SIN INFORMACIÓN,PACOA,70623,"4.596389,-74.074639"
6945,ESCUELA RURAL DE TOAKA,CO-VAU,COMUNIDAD DE TOAKA,,VAUPÉS,SIN INFORMACIÓN,PACOA,70624,"1.007644476,-71.25103723"


### leyendo indice cambium-devicedevices

Se lee la información de los dispositivos de red monitoreados por Cambium. En esta lectura no hay referencia de fechas ya que solo hay una ocurrencia por MAC de dispositivo de red.

* site_id es la llave para cruzar con cada centro de conexión.
* mac, IP y name son datos básicos del dispositivo.
* ap_group identifica los dispositivos como INDOOR u OUTDOOR

In [34]:
total_docs = 30000
try:
    response = es.search(
        index= parametros.cambium_d_d_index,
        body={
                    "_source": ["site_id","mac","ip","ap_group"]
                  , "query": {
                    "match_all": {}
                  }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]

    datos_dev = pd.DataFrame([x["_source"] for x in elastic_docs])
    
    
except:
    exit()

Se descartan registros con site_id vacios y se limpian los NaN del dataframe

In [35]:
datos_dev.dropna(subset=['site_id'])
datos_dev.fillna('', inplace=True)
datos_dev = datos_dev.drop(datos_dev[(datos_dev['site_id']=='')].index)
datos_dev.sort_values(['site_id','ap_group'], inplace=True)

Se limpian datos mal formados de ap_group

In [36]:
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("-", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("_", n = 1, expand = True)[0]
datos_dev = datos_dev.drop(datos_dev[(datos_dev['ap_group']=='')].index)

In [37]:
datos_dev = datos_dev.drop_duplicates('mac')

Se renombran campos según formato del indice final

In [38]:
datos_dev = datos_dev.rename(columns={'ap_mac' : 'centros.macRed','ap_group': 'centros.apGroup'
                                        , 'ip': 'centros.IP'
                                        , 'mac' : 'centros.macRed'
                                        , 'name' : 'centros.deviceName'})

In [39]:
datos_dev

,centros.apGroup,centros.IP,site_id,centros.macRed
3319,INDOOR,172.25.41.219,10003700,BC:E6:7C:5D:20:E1
3320,OUTDOOR,172.25.41.220,10003700,BC:A9:93:0E:2F:15
3321,OUTDOOR,172.25.41.221,10003700,BC:A9:93:0E:63:3E
3282,INDOOR,172.25.46.251,10501-ZZZY338,BC:E6:7C:ED:8A:FE
3283,OUTDOOR,172.25.46.253,10501-ZZZY338,BC:E6:7C:E8:3F:AE
...,...,...,...,...
1136,OUTDOOR,172.25.132.77,PRUEBA-LABORATORIO,BC:E6:7C:E7:FC:2B
1137,OUTDOOR,172.25.132.76,PRUEBA-LABORATORIO,BC:E6:7C:E7:FC:21
1477,INDOOR,172.25.117.11,TUMBATORO,BC:E6:7C:5F:3A:F1
1478,OUTDOOR,172.25.117.12,TUMBATORO,BC:E6:7C:E8:3F:8A


### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [40]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
               "_source": ["tablero06.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia-tablero06"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['tablero06.fechaControl']
except:
    fecha_ejecucion = '2021-05-01 00:00:00'
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-01 00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-06-25 00:00:00


## Se lee la información de cambium device performance

 Se toma los valores de dispositivos de red y su desempeño.
 * mac del dispositivo de red
 * timestamp es la fecha y hora de la medición
 * radio.* volumen de datos descargados(r) y cargados(t)

In [41]:
def traePerformance(fecha_max,fecha_tope):
    total_docs = 5000000
    response = es.search(
        index= parametros.cambium_d_p_index,
        body={
                "_source": ["mac","timestamp","radio.5ghz.rx_bps",
                           "radio.5ghz.tx_bps","radio.24ghz.rx_bps"
                          ,"radio.24ghz.tx_bps"]
              , "query": {
                  "range": {
                    "timestamp": {
                      "gte": fecha_max,
                      "lt": fecha_tope
                    }
                  }
              }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

### Lanzando  ejecución de consulta

* Se calcula rango en base a la fecha de control. Para este caso es de 10 minutos.
* Se ejecuta la función de consulta con el rango de fechas.
* Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha y hora actual.

In [42]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_performance = traePerformance(fecha_max_mintic,fecha_tope_mintic)

if datos_performance is None or datos_performance.empty:
    while (datos_performance is None or datos_performance.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")
        datos_performance = traePerformance(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

In [43]:
#datos_performance = traePerformance('2021-06-10 09:00:00','2021-06-10 10:00:00')

funcion para insertar en indice: 

In [44]:
use_these_keys = ['centros.nomCentroDigital',
                  'centros.codISO',
                  'centros.localidad',
                  'centros.siteID',
                  'centros.nombreDepartamento',
                  'centros.sistemaEnergia',
                  'centros.nombreMunicipio',
                  'centros.idBeneficiario',
                  'centros.apGroup',
                  'centros.IP',
                  'centros.deviceName',
                  'centros.macRed',
                  'centros.fechaControl',
                  'centros.traficoIN',
                  'centros.traficoOUT',
                  'centros.consumoGB',
                  'centros.fecha',
                  'centros.anyo',
                  'centros.mes',
                  'centros.dia',
                  'centros.hora',
                  'centros.minuto',
                  'nombreDepartamento',
                    'nombreMunicipio',
                    'idBeneficiario',
                    'fecha',
                    'anyo',
                    'mes',
                    'dia',
                  '@timestamp']
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'consumo-' + document['trafico.tablero06.siteID'] + '-' + document['trafico.tablero06.macRed'] + '-' +document['trafico.totales.tablero06.fechaControl']}",
                "_source": filterKeys(document),
            }

In [45]:
datos_dev

,centros.apGroup,centros.IP,site_id,centros.macRed
3319,INDOOR,172.25.41.219,10003700,BC:E6:7C:5D:20:E1
3320,OUTDOOR,172.25.41.220,10003700,BC:A9:93:0E:2F:15
3321,OUTDOOR,172.25.41.221,10003700,BC:A9:93:0E:63:3E
3282,INDOOR,172.25.46.251,10501-ZZZY338,BC:E6:7C:ED:8A:FE
3283,OUTDOOR,172.25.46.253,10501-ZZZY338,BC:E6:7C:E8:3F:AE
...,...,...,...,...
1136,OUTDOOR,172.25.132.77,PRUEBA-LABORATORIO,BC:E6:7C:E7:FC:2B
1137,OUTDOOR,172.25.132.76,PRUEBA-LABORATORIO,BC:E6:7C:E7:FC:21
1477,INDOOR,172.25.117.11,TUMBATORO,BC:E6:7C:5F:3A:F1
1478,OUTDOOR,172.25.117.12,TUMBATORO,BC:E6:7C:E8:3F:8A


## En otra  jerarquía se escriben los dispositivos conectados

Se toma el dataframe del proceso anterior para realizar el siguiente proceso:
* Se cuentan la cantidad de dispositivos WAN/LAN (OUTDOOR/INDOOR), agrupando por fecha y sitio
* Cantidad dispositivos WAN/LAN conectados se calcula validando cuando son OUTDOOR/INDOOR y tienen trafico(traficoOUT)
* Cantidad de dispositivos desconectados, restando los dos anteriores

Datos generados
* trafico.totales.cantDevWAN
* trafico.totales.cantDevLAN
* trafico.totales.cantDevConectadosWAN
* trafico.totales.cantDevDesconectadosWAN
* trafico.totales.cantDevConectadosLAN
* trafico.totales.cantDevDesconectadosLAN
* trafico.totales.cantDev

In [46]:
try:
    mintic_03 =['centros.nomCentroDigital',
                      'centros.codISO',
                      'centros.localidad',
                      'centros.siteID',
                      'centros.nombreDepartamento',
                      'centros.sistemaEnergia',
                      'centros.nombreMunicipio',
                      'centros.idBeneficiario',
                      'centros.location',
                      'centros.apGroup',
                      'centros.fechaControl',
                      'centros.fecha',
                      'centros.anyo',
                      'centros.mes',
                      'centros.dia',
                      'centros.hora',
                      'centros.minuto',
                      'centros.traficoIN',
                      'centros.traficoOUT',
                      'centros.consumoGB',]
    mintic_03.columns = mintic_03.columns.droplevel(1)
    mintic_03.rename(columns={'centros.traficoIN' : 'centros.traficoIN'
                          ,'centros.traficoOUT' : 'centros.traficoOUT'
                          ,'centros.consumoGB' : 'centros.consumoGB'
                         }, inplace=True)
except:
    pass

In [47]:
mintic_03

['centros.nomCentroDigital',
 'centros.codISO',
 'centros.localidad',
 'centros.siteID',
 'centros.nombreDepartamento',
 'centros.sistemaEnergia',
 'centros.nombreMunicipio',
 'centros.idBeneficiario',
 'centros.location',
 'centros.apGroup',
 'centros.fechaControl',
 'centros.fecha',
 'centros.anyo',
 'centros.mes',
 'centros.dia',
 'centros.hora',
 'centros.minuto',
 'centros.traficoIN',
 'centros.traficoOUT',
 'centros.consumoGB']

Funcion para insertar en indice la cantidad de dispositivos conectados 
* la lista use_these_keys se usa para referenciar cuales campos del dataframe irán al indice final. si los datos no se declaran en este, no se insertarán


In [48]:
use_these_keys = ['centros.nomCentroDigital',
                  'centros.codISO',
                  'centros.localidad',
                  'centros.siteID',
                  'centros.nombreDepartamento',
                  'centros.sistemaEnergia',
                  'centros.nombreMunicipio',
                  'centros.idBeneficiario',
                  'centros.location',
                  'centros.apGroup',
                  'centros.fechaControl',
                  'centros.cantDevWAN',
                  'centros.cantDevConectadosWAN',
                  'centros.cantDevDesconectadosWAN',
                  'centros.cantDevLAN',
                  'centros.cantDevConectadosLAN',
                  'centros.cantDevDesconectadosLAN',
                  'centros.cantDev',
                  'centros.cantDevConectados',
                  'centros.cantDevDesconectados',
                  'centros.fecha',
                  'centros.anyo',
                  'centros.mes',
                  'centros.dia',
                  'centros.hora',
                  'centros.minuto',
                  'centros.traficoIN',
                  'centros.traficoOUT',
                  'centros.consumoGB',
                  '@timestamp']

def doc_generator_dis(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'Conectados-' + document['centros.siteID'] + '-' +document['centros.fechaControl']}",
                "_source": filterKeys(document),
            }

# Insertando cantidad de dispositivos conectados

In [33]:
try:
    
mintic_03['@timestamp'] = now.isoformat()

salida = helpers.bulk(es, doc_generator_dis(mintic_03))

print("Fecha: ", now,"- Datos Dispositivos Conectados en indice principal:",salida[0])

except:
print("Fecha: ", now,"- No se insertaron datos de dispositivos conectados en indice principal")

IndentationError: expected an indented block (<ipython-input-33-34ce73491ca4>, line 3)

In [34]:
type(mintic_03)

pandas.core.frame.DataFrame

In [35]:
mintic_03 = pd.merge(mintic_03,  cantDevWAN, on=['centros.siteID','centros.fechaControl'],how='left')

mintic_03['@timestamp'] = now.isoformat()
salida = helpers.bulk(es, doc_generator_dis(mintic_03))
print("Fecha: ", now,"- Datos Dispositivos Conectados en indice principal:",salida[0])

KeyError: 'centros.siteID'

### Guardando fecha para control de ejecución

* Se actualiza la fecha de control. Si el calculo supera la fecha hora actual, se asocia esta ultima.

In [ ]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")[0:15] + '0:00'    

if fecha_ejecucion > str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia-conteos',
        body = { 'jerarquia-conteos': 'jerarquia-conteos','conteos.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)